# Jointure

D'une manière générale, le sujet des jointures en big data est un des sujets de ce domaine les plus complexes.

Le principe d'une jointure consiste, en effet, à fusionner deux datasets en recherchant dans les deux datasets les lignes qui partagent la même clé. Un algorithme naïf va résoudre une jointure en $O(n^2)$ : "pour chaque élément du premier dataset, je parcours le second dataset pour retrouver le ou les éléments paratageant la même clé". Dans un contexte big data, une telle complexité n'est pas envisageable, du fait de la taille des données à traiter et des communications réseau que cela peut engendrer.

Il existe heureusement des algorithmes de jointure bien plus efficaces, qui vont consister à utiliser une indexation sur la clé et/ou un tri sur la clé et/ou la diffusion d'un dataset entier sur les différents exécuteurs pour réaliser les opérations de recherche en local.

**De préférence, faites l'exercice en binôme.**

## Préambule

In [ ]:
import $ivy.`org.apache.spark::spark-core:3.3.2`
import $ivy.`org.apache.spark::spark-sql:3.3.2`
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`

import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

// Avoid disturbing logs
Configurator.setRootLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._
import java.net.URL

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("SparkSQL - Jointure")
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
    .getOrCreate()
}

import spark.implicits._

// Ce script fournit des fonctions supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper, spark_helper._

println(">>> Emplacement de Spark UI")
spark.sparkContext.uiWebUrl.foreach(url => println(s"  * $url"))
spark.sparkContext.uiWebUrl.map(new URL(_).getPort).map(port => s"http://localhost:$port").foreach(url => println(s"  * $url"))

## Chargement des datasets

Nous allons utiliser dans ce notebook des datasets différents de ce qui a été vu jusque-là. Nous nous mettons dans le cadre d'une application sur téléphone, qui permet à ses utilisateurs de partager sa position avec leurs amis. Pour cela, l'application détecte par géolocalisation le lieu et propose à l'utilisateur de partager le lieu (_checkin_). Une gamification est ajoutée, qui permet à l'utilisateur de gagner des badges selon la fréquence de ses checkins, la distance entre deux chekins, l'utilisateur qui effectue régulièrement les premiers checkins de la journée d'un même lieu devient "Maire" de ce lieu...

Nous avons deux datasets :

* Venues : représente un ensemble de lieux enregistrés qu'il est possible de visiter. Il contient notamment l'identifiant du lieu et ses coordonnées.
* Checkins : représente l'ensemble des checkins réalisés par les utilisateurs de l'application. Il contient l'identifiant de l'utilisateur, l'identifiant du lieu et le timestamp du checkin.

In [ ]:
import java.time.Instant

case class Venue(id: String, latitude: Double, longitude: Double, locationType: String, country: String)
case class Checkin(userId: String, venueId: String, timestamp: Instant)

val venuesFilename   = "data/threetriangle/venues.txt.gz"
val checkinsFilename = "data/threetriangle/checkins.txt.gz"

val venues =
  spark.read
    .option("sep", "\t")
    .schema("id STRING, latitude DOUBLE, longitude DOUBLE, locationType STRING, country STRING")
    .csv(venuesFilename)
    .as[Venue]

val checkins =
  spark.read
    .option("sep", "\t")
    // Option to correctly interpret timestamp in checkin data
    .option("timestampFormat", "EEE MMM d HH:mm:ss Z yyyy")
    .schema("userId STRING, venueId STRING, timestamp TIMESTAMP, tzOffset INT")
    .csv(checkinsFilename)
    .as[Checkin]

venues.createOrReplaceTempView("venues")
checkins.createOrReplaceTempView("checkins")

## Jointure

Nous voulons maintenant retrouver les informations des différents lieux où ont été effectués les checkins. Nous avons donc besoin de réaliser une jointure entre les datasets Checkins et Venues.

La méthode `.join()` est assez simple à utiliser. Elle s'applique sur un premier dataframe, puis vous spécifiez en paramètre le dataframe avec lequel vous créez une jointure, en indiquant éventuellement la relation de jointure et le type de jointure (inner, outer, left outer, right outer..., par défaut : inner).

In [ ]:
val data = checkins.join(venues, checkins("venueId") === venues("id"))
data.showHTML(limit=10, truncate=40)

Affichons le plan d'exécution de cette requête. Essayer de percevoir les différentes, dont celles qui concernent la jointure (Quelles optimisations sont apportées ? Y a-t-il des échanges de données ?)

Note : un plan d'exécution se lit du bas vers le haut pour suivre les différentes étapes de traitement dans l'ordre.

In [ ]:
data.explain()

Allez dans Spark UI et explorez l'onglet "SQL / DataFrame". Spark UI montre un DAG assez complexe pour la requête.

**Approche alternative**

Voici l'équivalent de notre requête, mais cette fois exprimée en SQL.

In [ ]:
val data =
  spark.sql("""
SELECT *
FROM
  checkins c INNER JOIN venues v ON (v.id = c.venueId)
""")

data.showHTML(limit=10, truncate=40)

Le plan d'exécution de cette requête est-il similaire au plan précédent ?

In [ ]:
data.explain()

**Ce qu'il faut voir**

Dans le processus de génération du plan d'exécution physique, nous pouvons voir que Spark fini par sélectionner une stratégie de jointure. Cette stratégie se traduit par une phase d'échange (ou plus exactement, de diffusion) de données entre exécuteurs (`BroadcastExchange`). Puis, interviens la phase de jointure basée sur le _hash_ de la clé de jointure (`BroadcastHashJoin`).

Nous pouvons voir aussi que Spark se laisse la possibilité au dernier moment de changer de stratégie de jointure (`AdaptiveSparkPlan isFinalPlan=false`), sur la base de données statistiques récupérées pendant l'exécution.

Ce type de jointure est similaire à ce que nous avons vu avec les variables broadcast dans le cas des RDD.

## Stratégie de jointure

Spark SQL dispose de différentes stratégies de jointure. Nous venons d'en voir une dans la section précédente.

Sans précision dans le code, cette stratégie est choisie selon une heuristique paramétrable liée, notamment, au type de la jointure, la condition sur les clés (équivalence ou non-équivalence), à taille des données ou à d'autres données statistiques.

Si vous voulez forcer la stratégie de jointure, vous pouvez le préciser dans le code à travers des _hints_ :

```scala
  df1.join(df2.hint("<Stratégie>"), ...)
```

En SQL :

```scala
  spark.sql("""SELECT /*+ <Stratégie> */ ...""")
```

Voici les différentes stratégies (et les valeurs à utiliser dans les hints) :

 * Broadcast Hash Join (BROADCAST / BROADCASTJOIN / MAPJOIN)
 * Shuffle Sort-Merge Join (MERGE / SHUFFLE_MERGE / MERGEJOIN)
 * Shuffle Hash Join (SHUFFLE_HASH)
 * Shuffle-and-Replicate Nested Loop Join (SHUFFLE_REPLICATE_NL)

### Broadcast Hash Join

Cette stratégie est utilisée dans ces conditions :
 * La relation de jointure se base sur l'égalité entre les clés (ie. `dataset1("key") === dataset2("key")`).
 * L'un des datasets est considéré comme étant suffisamment petit.
 * Tous les types de jointure sont supportés, sauf FULL OUTER JOIN.

Dans ce cas, le "petit" dataset est transmis (ou diffusé, d'où le terme _broadcast_) sous la forme d'une table de hachage (_hash table_) à l'ensemble des exécuteurs participant à la jointure. La jointure est alors réalisée en local sur chaque nœud.

Le seuil indiquant si la stratégie sera utilisée est fixée par le paramètre `spark.sql.autoBroadcastJoinThreshold`. Il est exprimé en octets (valeur par défaut : 10485760 (= 10 MB)). Si l'un des datasets de la jointure à une taille inférieure à ce seuil, il sera diffusé. Si aucun 

Dans le cadre de la fonctionnalité AQE (_Adaptive Query Execution_), qui permet de pousser Spark à sélectionner une autre startégie sur la base de relevés statistiques, au lieu de la stratégie planifiée initialement, le seuil permettant de passer à la stratégie _Broadcast Hash Join_ est fixé par le paramètre `spark.sql.adaptive.autoBroadcastJoinThreshold` exprimé en octet. Si vous fixez sa valeur à -1, cette fonctionnalité est désactivée.

Un autre paramètre intervient : `spark.sql.broadcastTimeout`. Ce paramètre est exprimé en secondes (valeur par défaut : 300 (= 5mn)). Il est détecté des problèmes de communication durant la diffusion des données. Si la diffusion de la table ne peut pas être terminée dans le délai imparti, Spark interrompt l'exécution de la requête et génère une erreur.

### Shuffle Sort-Merge Join

Cette stratégie est utilisée dans ces conditions :
 * La relation de jointure se base sur l'égalité entre les clés (ie. `dataset1("key") === dataset2("key")`).
 * Il est possible de trier les clés.
 * Tous les types de jointure sont supportés.
 
C'est la stratégie utilisée par défaut (sauf dans le cas ou le paramètre `spark.sql.join.preferSortMergeJoin=false`, dans ce cas, Spark utilisera _Shuffle Hash Join_).

Les étapes de jointure sont :
 1. Échanges des données entre exécuteurs en fonction de la clé.
 2. Tri des données en local en fonction de la clé.
 3. Fusion des deux datasets.

En général, _Shuffle Sort-Merge Join_ a de meilleures performances sur des données volumineuses.

### Shuffle Hash Join

Cette stratégie est utilisée dans ces conditions :
 * La relation de jointure se base sur l'égalité entre les clés (ie. `dataset1("key") === dataset2("key")`).
 * Il est possible de trier les clés.
 * Tous les types de jointure sont supportés, sauf FULL OUTER JOIN.

Les étapes de jointure sont :
 1. Échanges des données entre exécuteurs en fonction de la clé.
 2. Dépôt des données transférées en local dans une table de hachage en fonction de la clé.
 3. Fusion des deux datasets.

_Shuffle Hash Join_ est efficace lorsque la clé utilisée pour la jointure permet un partitionnement équilibré de la donnée.

### Shuffle-and-Replicate Nested Loop Join

Cette stratégie applique l'algorithme naïf vu plus haut : "pour chaque élément du premier dataset, je parcours le second dataset pour retrouver le ou les éléments partageant la même clé".

Ce type de jointure convient lorsque :
 * Le premier dataset est diffusable sur une jointure de type RIGHT OUTER JOIN.
 * Ou le second dataset est diffusable sur des jointures de type LEFT OUTER JOIN, LEFT SEMI JOIN ou LEFT ANTI JOIN.
 * Ou Dans tous les autres cas sur des jointures de type INNER JOIN ou équivalent.

### Produit cartésien

Le produit cartésien (ou CROSS JOIN) consiste à retourner toutes les combinaisons possibles de lignes sur la fusion de deux dataset.

Ce type de jointure intervient lorsque :
 * Le type de jointure est INNER JOIN ou équivalent.
 * Ou `cross` est explicitement indiqué comme type de jointure
 
Dans les autres cas, l'opération de jointure renvoie une erreur de type AnalysisException.

### Exercices

Pour chacun des exercices ci-dessous, vous devez :
 1. Exécuter la cellule contenant la requête.
 2. Observer le plan d'exécution
 3. Retrouver et observer dans Spark UI le DAG de la requête correspondante
 
Essayez de distinguer les différences qu'il peut y avoir entre les diverses stratégies.

### Broadcast Hash Join

In [ ]:
val data = checkins.join(venues.hint("BROADCAST"), checkins("venueId") === venues("id"))
data.showHTML(limit=10, truncate=40)
data.explain()

### Shuffle Sort-Merge Join

In [ ]:
val data = checkins.join(venues.hint("MERGE"), checkins("venueId") === venues("id"))
data.showHTML(limit=10, truncate=40)
data.explain()

### Shuffle Hash Join

In [ ]:
val data = checkins.join(venues.hint("SHUFFLE_HASH"), checkins("venueId") === venues("id"))
data.showHTML(limit=10, truncate=40)
data.explain()

### Shuffle-and-Replicate Nested Loop Join

Pour commencer, nous allons utiliser le hint `SHUFFLE_REPLICATE_NL` pour impliquer implicitement un produit cartésien.

In [ ]:
val data = checkins.join(venues.hint("SHUFFLE_REPLICATE_NL"), checkins("venueId") === venues("id"))
data.showHTML(limit=10, truncate=40)
data.explain()

Recherchons les lieux qui n'ont pas encore reçu de visite des utilisateurs. Pour cela, nous allons utiliser une jointure de type LEFT ANTI JOIN.

In [ ]:
val data = venues.join(checkins.hint("SHUFFLE_REPLICATE_NL"), checkins("venueId") === venues("id"), "leftanti")
data.showHTML(limit=10, truncate=40)
data.explain()

### Jointure sans clé

Cette fois, sans préciser la clé, nous allons générer toutes les combinaisons possibles d'utilisateurs et de lieux.

In [ ]:
val data = checkins.select($"userId").join(venues)
data.showHTML(limit=10, truncate=40)
data.explain()